In [43]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [44]:
# 1. Load the dataset
df = pd.read_csv('data.csv')
print(df.head())

  employee_id  performance  years_of_experience  projects_completed  bonus
0     EMP_001            7                    2                   4    124
1     EMP_002            4                    1                   4     82
2     EMP_003            8                    7                  10    178
3     EMP_004            5                    7                   8    138
4     EMP_005            7                    8                   9    170


In [45]:
# 2. Define Features and Target
X = df[["performance", "years_of_experience", "projects_completed"]].values
y = df[["bonus"]].values  # Make sure y is 2D for scaling

In [46]:
# 3. Scale the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [47]:
# 4. Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=0)

In [48]:
# 5. Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [49]:
# 6. Define the model
class BonusPrediction(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(3, 1)
        )

    def forward(self, x):
        return self.network(x)

model = BonusPrediction()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [50]:
# 7. Training loop
epochs = 1000
for epoch in range(epochs):
    # Forward pass
    y_pred = model(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch: [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch: [100/1000], Loss: 1.1805
Epoch: [200/1000], Loss: 1.0840
Epoch: [300/1000], Loss: 1.0172
Epoch: [400/1000], Loss: 0.9710
Epoch: [500/1000], Loss: 0.9391
Epoch: [600/1000], Loss: 0.9169
Epoch: [700/1000], Loss: 0.9015
Epoch: [800/1000], Loss: 0.8908
Epoch: [900/1000], Loss: 0.8834
Epoch: [1000/1000], Loss: 0.8783


In [51]:
# 8. Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)

print(f'\nTest Loss: {test_loss.item():.4f}')


Test Loss: 0.7862


In [52]:
y_pred_original = scaler_y.inverse_transform(y_pred.numpy())
y_test_original = scaler_y.inverse_transform(y_test_tensor.numpy())

print("\nPredicted Bonus (original scale):")
print(y_pred_original[:5])
print("\nActual Bonus (original scale):")
print(y_test_original[:5])



Predicted Bonus (original scale):
[[278.73685]
 [237.86403]
 [192.58945]
 [362.76752]
 [286.64008]]

Actual Bonus (original scale):
[[170.]
 [248.]
 [178.]
 [307.]
 [335.]]
